<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Azure Managed Disks

## Azure Actions Covered

* Creating virtual machines


In this lecture, we're going to take a look at how to create virtual machines via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add some new imports as well.

In [2]:
from azure.identity import AzureCliCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute.models import DiskCreateOption

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

In [4]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [5]:
compute_client = ComputeManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [6]:
poller = compute_client.disks.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'empty-managed-disk',
    {
        'location': DEFAULT_LOCATION,
        'disk_size_gb': 20,
        'creation_data': {
            'create_option': DiskCreateOption.empty
        }
    }
)
disk_resource = poller.result()

In [7]:
disk_resource.provisioning_state

'Succeeded'

In [9]:
vm = compute_client.virtual_machines.get(DEFAULT_RESOURCE_GROUP, 'test-vm')

In [10]:
managed_disk = compute_client.disks.get(DEFAULT_RESOURCE_GROUP, 'empty-managed-disk')

In [11]:
vm.storage_profile.data_disks.append({
    'lun': 12,
    'name': managed_disk.name,
    'create_option': DiskCreateOption.attach,
    'managed_disk': {
        'id': managed_disk.id
    }
})

In [12]:
async_update = compute_client.virtual_machines.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    vm.name,
    vm
)
async_update.wait()